In [ ]:
%%R


In [ ]:
from preamble import *
from stat_tools import *
from scikit_tools import *
import scipy
plt.close('all')

In [ ]:
from Bachelier import *

# Kernel methods for optimal transportation

## Discrete ordering algorithms

We describe in this section discrete ordering algorithms based on the kernel-based distance.

### Linear Sum Assignment Problems (LSAP)

In this section, we describe a ordering algorithm, which we discovered while working in the development of algorithms based on the theory of Reproducing Kernel Hilbert Space (RKHS). This algorithm is motivated by the ``linear assignment value'' problem, and is used in a number of our Academic and industrial applications; 

The Linear Sum Assignment problem (LSAP) is a fundamental problem of combinatorial optimization. It is an old and well-documented problem \footnote{see the wikipedia page \url{https://en.wikipedia.org/wiki/Assignment_problem}}, which leads to a large number of important industrial applications. It has been solved in the early 30's by H.W. Kuhn and is often called the Hungarian method in order to highlight that it derives from two older\footnote{These algorithms might be credited to Jacobi posthumous papers \cite{CGJacobi:1890}.} results by two Hungarians mathematicians, Koenig (Math Ann 77:453 465, 1916) and Egervry (Mat Fiz Lapok 38:1628, 1931). In particular, the approach adopted in the present text relies on the LSAP. As our library embeds a different algorithm than the Hungarian one for performance purposes, we describe and exhibit our interface on this problem in the next section.

Our basic idea here is to use a ordering algorithm and order any two sets of points $x \in \RR^{N_x \times D}$, $y \in \RR^{N_y \times D}$ with respect to the *discrepancy distance matrix* $d_k(x,y) \in \RR^{N_x \times N_y}$ associated with a given kernel, introduced in \@ref(eq:norm). Without loss of generality, suppose that $N_x > N_y$. The above distance allows us to compute a permutation $\sigma$ with length $N_y$, with the help of the LSAP algorithm. Then, we reorder and output the distribution $x$ with this permutation. Our motivation comes from optimal transportation; see for instance \cite{Villani:2009} for a review of optimal transport. Indeed, once computed, the map
$$
  x^n \mapsto y^{\sigma_n}
$$
defines an optimal map, with respect to the kernel-induced *discrepancy distance* $d_k(x,y)$, described in section \@ref(Discrepancy-error), transporting the measure $\mu_x := \sum_n \delta_{x^n}$ into the measure $\mu_y := \sum_n \delta_{y^{\sigma_n}}$. This equivalence between the LSAP and the Monge-Kantorovich problem, used since some time, has only been recently rigorously proven in \cite{Brezis:2018}.

#### Description of the problem

Linear sum assignment problems can be well described using graph theory : it consists of finding, in a weighted bipartite graph, a matching of a given size, in which the sum of weights of the edges is a minimum.

To make this definition clear, let us introduce some notations : consider any real-valued matrix $M \in \RR^{N_x \times N_y}$, called a *cost* matrix. A linear assignment problem consist in finding a permutation $\sigma : [1 \ldots \min(N_x,N_y)] \mapsto [1 \ldots \min(N_x,N_y)]$ such that
$$
  \sigma = \arg \inf_{\sigma \in \Sigma} \sum_{n \le \min(N_x,N_y)} M(n,\sigma(n)),
$$
where $\Sigma$ holds here for the set of all permutations.
Another equivalent formulation is the following one
$$
  \sigma = \arg \inf_{\sigma \in \Sigma} \sigma \cdot M
$$
where $A\cdot B$ holds here for the Frobenius scalar product and $\Sigma$ is the set of permutations, using a matrix representation : $\sigma \in \RR^{N_x \times N_y}$ $\sum_n \sigma(n,m) = \sum_m \sigma(n,m) = 1, \sigma(n,m) \in \{0,1\}$.

Let us give a quick illustration for better understanding to this problem. We fill out a matrix with random values in table \@ref(tab:512), and output also its cost, that is $Tr(M)$.

In [ ]:
def cost(M):
  cost = 0.
  for n in range(len(M)): cost += M[n,n]
  return cost
  
N = 4
M = np.random.rand(N,N)
M_df = pd.DataFrame(M)
print("total cost:",cost(M))

In [ ]:
%%R
knitr::kable(py$M_df, caption = "a 4x4 random matrix")

Then we compute the permutation $\sigma$. The python interface to this function is simply $\sigma = \text{lsap}(M)$.

In [ ]:
permutation = alg.lsap(M)
print("permutation: ",permutation)

We reorder $\tilde{M} = \sigma M$, and we ouput the new cost after ordering, that is $Tr(\tilde{M})$. we check in the following that the lsap algorithm decreased the total cost.

In [ ]:
M = M[permutation]
print("total cost:",cost(M))

#### Description of the algorithm

Our ordering algorithm is quite straightfoward. Consider any two set of points $x \in \RR^{N_x \times D}$, $y \in \RR^{N_y \times D}$, a kernel $k(x,y)$, and the \textit{discrepancy} distance $d_k(x,y) \in \RR^{N_x \times N_y}$ introduced in \@ref{distance-matrices}. Suppose wlog $N_y < N_x$. We compute first the  permutation
$$
  \sigma = \text{lsap}(d_k(x,y)), \quad \sigma \in \RR^{N_y}
$$

Then output the reordered set
$$
  x^\sigma := (x^{\sigma_1},\ldots,x^{\sigma_{N_y}}), \quad y,\quad \text{ if }N_x \ge N_y
$$

#### Description of the python function

The ordering algorithm takes two distributions in input, and output a permutation of one of its input data ($x$ or $y$), as well as the permutation $\sigma$:

$$
  x^\sigma,y^\sigma,\sigma = alg.reordering(x,y,set\_codpy\_kernel, rescale,distance = None)
$$

This algorithm takes in input the following:

* Two distributions of points having shapes
$$
  x := (x^1,\ldots,x^{N_x}) \in \mathbb{R}^{N_x \times D}, \quad y:=(y^1,\ldots,y^{N_y}) \in \mathbb{R}^{N_y \times D}
$$
* A positive kernel $k(x,y)$, defined through the input variable set\_codpy\_kernel. This defines the cost matrix as being $M = d_k(x,y)$, where the distance matrix is defined in \@ref(distance-matrices).

* Alternatively an optional parameter $distance$ taking values among
  * "norm1", in which case the sorting is done accordingly to the Manhattan distance $d(x,y) = |x-y|_1$
  * "norm2", in which case the sorting is done accordingly to the Euclidean distance $d(x,y) = |x-y|_2$
  * "normifty", in which case the sorting is done accordingly to the sup-distance $d(x,y) = |x-y|_\infty$

This function outputs :

* Two distributions $x^\sigma,y^ \sigma$ having length $N_y$. If $N_x > N_y$, then $y^\sigma=y$.
The case $N_y>N_x$ is symetrical, letting the original distribution $x$ unchanged.
* A permutation $\sigma$, represented as a vector $i \mapsto \sigma_i$, $0 \le i \le \min(N_x,N_y)$.

#### Illustration for the square matrix case


##### A quantitative illustration

We show first the results given by our ordering algorithm on a simple example. We generate two distributions of 4 points in $\mathbb{R}^{5}$. The first is generated by multivariate Gaussian distribution centered in $(5,..,5)$, the second one by a uniform distribution supported into the unit cube.

In [ ]:
N = 4
D = 5
x0,y0 = np.random.normal(5., 1., (N,D)),np.random.rand(N,D)
x_df,y_df = pd.DataFrame(x0),pd.DataFrame(y0)

We output x in table \@ref(tab:679) and y in table \@ref(tab:680)

In [ ]:
%%R
knitr::kable(py$x_df, caption = "a random gaussian distribution x")

In [ ]:
%%R
knitr::kable(py$y_df, caption ="a random uniform distribution y")

Let us first pick up a kernel $k$, here a Matern kernel.

In [ ]:
set_codpy_kernel = kernel_setters.kernel_helper(kernel_setters.set_matern_tensor_kernel,2,1e-8,map_setters.set_mean_distance_map)

Then we compute the distance matrix, and output the transportation cost $\sum_{n=0}^N d_k(n,n)$

In [ ]:
Dnm = op.Dnm(x0,y0,set_codpy_kernel = set_codpy_kernel,rescale=True)
Dnm_df = pd.DataFrame(Dnm)
print("cost:", cost(Dnm))

We output the distance matrix in table \@ref(tab:681).

In [ ]:
%%R
knitr::kable(py$Dnm_df)

We then invoke the ordering algorithm and output the cost after ordering. 

In [ ]:
x,y,permutation = alg.reordering(x0,y0,set_codpy_kernel = None, rescale = False)
Dnm = op.Dnm(x,y,set_codpy_kernel = None, rescale = False)
Dnm_df = pd.DataFrame(Dnm)
print("cost:", cost(Dnm))
permutation = np.asarray(permutation)[0:4].reshape(1,4)

Finally, we output the distance matrix again after ordering in table \@ref(tab:682), as well as the permutation $\sigma$ in table \@ref(tab:683)

In [ ]:
%%R
knitr::kable(py$Dnm_df,caption = "Matrix after ordering (samples)")

In [ ]:
%%R
knitr::kable(py$permutation,caption = "permutation")

One can check that the sum of the diagonal elements has decreased.

##### A qualitative illustration

This algorithm can be best illustrated in the two-dimensional case.

We first consider the Euclidean distance function $d(x,y) = |x-y|$, in which case this algorithm  corresponds to a classical rearrangement, i.e. the one corresponding to the Wasserstein distance. To illustrate this behavior, let us generate a bi-modal type distribution $x \in \RR^{N \times D}$ and a random uniform one $y \in [0,1]^{N \times D}$. 


In [ ]:
N=16
D = 2
left = np.random.normal(-3., 1., (int(N/2),D))
right = np.random.normal(3., 1., (int(N/2),D))
x0 = np.concatenate( (left,right) )
y0 = np.random.rand(len(x0),D)

For a convex distance, this algorithm is characterized by a ordering where characteristic lines do not cross each others, as plot in the picture \@ref(fig:185), plotting both edges $x^i \mapsto y^i$, before and after the ordering algorithm.

In [ ]:
x,y,permutation = alg.reordering(x=x0,y=y0, set_codpy_kernel = None, distance = "norm2")
reordering_plot(x0,y0,x,y)

Note however that kernels based distance might lead to different permutations. This is due to the fact that kernels defines distance that might not be euclidean. Indeed, kernel distance might not respect the triangular inequality. For instance, the kernel selected above defines a distance equivalent to $d(x,y) = \Pi_d |x_d-y_d|$, and leads to a ordering for which some characteristics should cross

In [ ]:
x,y,permutation = alg.reordering(x=x0,y=y0, set_codpy_kernel = set_codpy_kernel, rescale = True)
reordering_plot(x0,y0,x,y)

### LSAP extensions

In this section we describe some extensions of the LSAP algorithms that we use in our library.

#### Different input sizes

A first quite straightforward extension of LSAP problem can be found for inputs set of different sizes, wlog $N_y\le N_x$. The figure \@ref(fig:184) illustrates the behavior of our LSAP algorithm in this setting

In [ ]:
N=32
M=16
D = 2
left = np.random.normal(-3., 1., (int(N/2),D))
right = np.random.normal(3., 1., (int(N/2),D))
x0 = np.concatenate( (left,right) )
y0 = np.random.rand(M,D)
x,y,permutation = alg.reordering(x=x0,y=y0, set_codpy_kernel = None, distance = "norm2")
reordering_plot(x0,y0,x,y)

#### General cost functions and motivations

Consider any  real-valued matrix $M \in \RR^{N \times N}$. In situations of interests, we consider cost functional $c(M)$ that generalizes the classical cost functional for LSAP problem $c(M) = \sum_{n} M(n,n)$. Our algorithm generalizes to these cases, finding a permutation $\sigma : [1 \ldots N] \mapsto [1 \ldots N]$ such that

$$
  \bar{\sigma} = \arg \inf_{\sigma \in \Sigma} c( M^\sigma),\quad M^\sigma = m(n,\sigma(n)) 
$$

An example of such a LSAP problem extension arised with kernel methods in section \@ref(sharp-discrepancy-sequences). It corresponds to compute the minimum of the discrepancy functional \@ref{eq:dk}, for the particular choice where $x^\sigma \subset x$ is a subset of $x$ having length $N_y<N_x$. We used the notations $x^\sigma=(x^{\sigma_1},\ldots,x^{\sigma_{N_y}})$, with $\sigma : [1\ldots N_y] \mapsto [1\ldots N_x]$. In this context, the matrix is defined as $M(n,m) = k\big(x^n,x^m\big)$, and the cost function is

$$
d_k\big(x,x^\sigma\big)^2 = c(M) = \frac{1}{N_x^2}\sum_{n=1,m=1}^{N_x,N_x} M(n,m) + \frac{1}{N_y^2}\sum_{n=1,m=1}^{N_y,N_y} M(\sigma(n),\sigma(m)) - \frac{2}{N_x N_y}\sum_{n=1,m=1}^{N_x,N_y} k\big(n,\sigma(m)\big).
$$

So that our target minimization problem can be described as finding a permutation $\bar{\sigma}$ such that

$$
 \overline{\sigma} = \arg \inf_{\sigma : [1\ldots N_y] \mapsto [1\ldots N_x]} c(M^\sigma\big),\quad M^\sigma(n,m) = k(x^n,x^{\sigma(m)})
$$

## Conditional expectation algorithm

### Introduction

In this section, we propose a general interface to a python function computing conditional expectations problems in arbitrary dimensions, that we named Pi. We also propose a kernel-based implementation of these problems, which algorithm is described in \cite{LeFloch-Mercier:2017} - \cite{LeFloch-Mercier:2020b}. 

Kernel methods to compute conditional expectations started to be considered a decade ago, see for instance \cite{Mercier:2014}. Indeed, these algorithms are centrals, particularly for finance applications, as they are the heart of pricing technologies. They also have numerous other applications. 

Benchmarking such algorithms is a difficult task, as the literature did not provide competitor algorithms to compute conditional expectations to kernel-based methods, for arbitrary dimensions, to our knowledge. Indeed, these algorithms are tightly concerned with the so called \textit{curse of dimensionality}, as we are dealing with arbitrary dimensions algorithms. 

However, there is a recent, but impressively fast-growing, literature, devoted to the study of Artificial Intelligence methods (AI), particularly for Finance applications, see \cite{GPW} and ref. therein for instance. In particular, a Neural Networks (NN) approach has been proposed to compute conditional expectation in \cite{NS} that we can use as benchmark. Hence a first benchmark is conducted in section \ref{the-bachelier-problem}.

### The Pi function

Consider any martingale process $t \mapsto X(t)$, and any positive definite kernel $k$, we define the operator $\Pi$ - using python notations -

\begin{equation}\label{Pi}
  f_{z | x} = \Pi(x,z,f(z)=[])
\end{equation}

where 

- $x \in \RR^{ N_x \times D}$ is any set of points generated by a i.i.d sample of $X(t^1)$ where $t^1$ is any time.

- $z \in \RR^{ N_z \times D}$ is any set of points, generated by a i.i.d sample of $X(t^2)$ at any time $t^2>t^1$.

- $f(z) \in \RR^{ N_z \times D_f}$ is any, optional, function, representing payoff values. 

The output is

- if $f(z)$ is let empty, the output $f_{z | x} \in \RR^{ N_z \times N_x}$ is a matrix, representing a convergent approximation of the stochastic matrix $\EE^X(z | x)$.

- if $f(z) \in \RR^{ N_z \times D_f}$ is not empty, $f_{z | x} \in \RR^{ N_z \times D_f}$ is a matrix, representing the conditional expectation $f(z |x) := \EE^X( f(z) | x)$.

## Polar factorization algorithms

Consider any mapping $S : \RR^D \mapsto \RR^D$, and a distance function, that is positive, scalar valued, $\mathcal{C}^1$ function $d(\cdot,\cdot)$.

The polar factorization algorithm amounts to find a **scalar, convex** function $f$, and a volume preserving map $T:\RR^D \mapsto \RR^D$, satisfying

$$
  S = \Big(\nabla f\Big)\circ T(y),\quad f \text{ convex}, \quad T_\#m = m (\#eq:PF)
$$
A volume preserving map is a mapping satisfying $\int \varphi dx = \int \varphi\circ T dx$ for any continuous function $\varphi$, $dx$ being the Lebesgue measure. Existence and unicity of this decomposition is discussed in Brenier seminal's paper \cite{BY}.

### Discrete Polar factorization and linear sum assignment problem

Let us start from any distributions $x \in \RR^{N_x \times D}$, $z \in \RR^{N_y \times D}$, and consider a distance function, that is positive, scalar valued, $\mathcal{C}^1$ function $d(\cdot,\cdot)$, and we naturally extend this function to a matrix valued one $d(x,z) \in \RR^{N_x \times N_z}$.
Let us first make some reminder about optimal transportation type problems.

### The Monge-Kantorovitch problem

Polar factorization are linked to the Monge-Kantorovitch problem, that is an optimal transportation one. Consider any two measures $\mu_x$, $\mu_z$, and consider a distance function, that is positive, symmetrical, convex, scalar valued, $\mathcal{C}^1$ function $d(\cdot,\cdot)$. 

The following discrete problem is called the **Monge** problem
$$
\bar{\gamma} = \inf_{\gamma \in \Gamma} d(x,z)\cdot \gamma  (\#eq:MO)
$$
where 

* $A \cdot B$ denotes the Frobenius scalar matrix product
* $\Gamma$ denotes the set of all bi-stochastic matrix $\gamma \in \RR^{N \times N}$, that is satisfying,
$$
\sum_{n = 1\ldots N} \gamma_{m,n} =  \sum_{n = 1\ldots N} \gamma_{n,m} = 1, \quad \gamma_{n,m}\ge 0, \quad \text{for all } n,m = 1,\ldots,N.
$$

This minimization problem owns a dual expression, called the **Kantorovitch** problem

$$
 \sup_{\varphi, \psi } \sum_n^N \varphi(x^n) - \psi(z^n),\quad \varphi(x^n) - \psi(z^n) \le d(x^n,z^n)  (\#eq:KA)
$$
where $\varphi,\psi$ are the unknown functions.

Note that any permutation $\sigma:[1\ldots N]\mapsto [1\ldots N]$ is a stochastic matrix. In particular, the following discrete problem, that is a Linear assignment problem, described in section \@ref(linear-sum-assignment-problems-lsap), consists in a first approach to \@ref(eq:MO):

$$
  \bar{\gamma} = \inf_{\gamma \in \Sigma} d(x,z)\cdot \gamma  (\#eq:MKII)
$$
Indeed, all problems \@ref(eq:MO)-\@ref(eq:MKII)-\@ref(eq:KA) are equivalent, see \cite{Brezis:2018}.

### Motivation: the sampler function

In many applications we would like to fit the scattered data to a given model that best represents them. To be specific, consider any distributions of points $x \in \RR^{N \times D}$, representing i.i.d. samples of a random variable $X$, $z \in \RR^{[0,1]^{N \times D}}$, any i.i.d. of the uniform distribution into the unit cube, and suppose that we solved \@ref(eq:PF) in the following, discrete, sense

$$
  x = \Big(\nabla f\Big)(z),\quad f \text{ convex}, \quad x \in \RR^{N \times D}, z \in [0,1]^{N \times D}.  (\#eq:fxz)
$$

Then the function 
$$
  y \mapsto \Big(\nabla f\Big)(y), (\#eq:fy)
$$
where $y \in \RR^{[0,1]^{N_y \times D}}$ provides us with a natural candidate for others i.i.d. realization of the random variable $X$.

Hence this section illustrates the following python function
$$ 
    y = sampler(x,M, seed) (\#eq:sampler)
$$
that outputs $M$ values $y \in \mathbb{R}^{N\times D}$ of a distribution sharing close statistical properties with the discrete distribution $x$, that we discuss in the next paragraph.

#### Statistical tests

We exhibit three statistical indicators to support our claims, measuring each some kind of distance between the two distributions $x$ and $y$. The two first tests are one-dimensional based tests. We check it on every axes. The third one is based on the discrepancy error.

* Kolmogorov-Smirnov based tests. These are one-dimensional tests, based on the cumulative distribution function. The test is
$$
  \|cdf_x  - cdf_z\|_\ell^\infty(\RR^N) \ge \frac{c_N}{\sqrt{N}} 
$$
where $c_N$ is a confidence level.
* Hellinger distance tests. To measure the closedness of $pdf_x$ to $pdf_z$ which are the PDFs of $x$ and $y$ repectively. The Hellinger distance is defined as
$$
\mathcal{H}(x,z) = \frac{1}{\sqrt{2}}||\sqrt{pdf_x} - \sqrt{pdf_z}||_2
$$
* Discrepancy errors, defined in section \@ref(discrepancy-error).

### One dimensional Examples

#### Bimodal Gaussian distribution

In this section we study a bi-modal gaussian distribution.


In [ ]:
(N,D) = (1000,1)
x = np.random.normal(-5., 1., (int(N/2),D))
x = np.concatenate( (x,np.random.normal(+5., 1., (int(N/2),D))) )

We output some values of $x$ in the following lines

In [ ]:
%%R
knitr::kable(head(py$x), caption = 'one-dimensional bi-modal distribution', label = 'Bi1D', col.names = c('samples bimodal distribution'))

Let us call the sampling function, filling up $y \in \mathbb{R}^{M\times 1}$.

In [ ]:
M = 500
y = alg.sampler(x,M)

We output some values of $y$ in the following lines 

In [ ]:
%%R
knitr::kable(head(py$y), caption = 'sampled distribution', label = 'Bi1DGen', col.names = c('generated samples'))

To check our results, let us compare both cdf of $x$ and $y$ in the following figure
<!--

In [ ]:
compare_plot(x,y,title = "original distribution (blue) versus generated distribution (yellow)")

-->

In [ ]:
%%R
plot(ecdf(py$x), verticals=TRUE, do.points=FALSE, main = "original versus generated CDFs (Gaussian bi-modal)", 
     col = 'black', lwd = 2, xlab = "x, y", ylab = "F(x), F(y)")
plot(ecdf(py$y), verticals=TRUE, do.points=FALSE, add=TRUE, col='blue', lwd = 2, lty = 2)
legend(1,0.4, legend = c("Original (x)", "Generated (y)"), col = c("black", 'blue'), lty =1:2, lwd = 2) 

In [ ]:
summary_ = summary((x,y))
ks_ = ks_testD(x,y)
#dd = compare_distances(x,y)

To check numerically some first properties of the generated distribution, We output in the following table the skewness and kurtosis of both $x$ and $y$

In [ ]:
%%R
knitr::kable(head(py$summary_), caption = 'distributions characteristics', label = 'Ga1Dsum')

In [ ]:
%%R
knitr::kable(py$ks_, caption = 'KS Test', label = 'BiKS')

#### Bimodal t-distribution

In this section we study a bi-modal t - distribution.


In [ ]:
(N,D, df) = (1000,1,3)
x = np.concatenate( (scipy.stats.t.rvs(df, -5., 1.,  (N,D)), 
scipy.stats.t.rvs(df, 5., 1., (N,D))) )

We output some values of $x$ in the following lines

In [ ]:
%%R
knitr::kable(head(py$x), caption = 'one-dimensional bi-modal distribution', label = 'Bi1D', col.names = c('samples bimodal distribution'))

Let us call the sampling function, filling up $y \in \mathbb{R}^{M\times 1}$.

In [ ]:
M = 500
y = alg.sampler(x,M)

We output some values of $y$ in the following lines 

In [ ]:
%%R
knitr::kable(head(py$y), caption = 'sampled distribution', label = 'Bi1DGen', col.names = c('generated samples'))

To check our results, let us compare both cdf of $x$ and $y$ in the following figure
<!--

In [ ]:
compare_plot(x,y,title = "original distribution (blue) versus generated distribution (yellow)")

-->

In [ ]:
%%R
plot(ecdf(py$x), verticals=TRUE, do.points=FALSE, main = "original versus generated CDFs (bi-modal t distribution)", 
     col = 'black', lwd = 2, xlab = "x, y", ylab = "F(x), F(y)")
plot(ecdf(py$y), verticals=TRUE, do.points=FALSE, add=TRUE, col='blue', lwd = 2, lty = 2)
legend(1,0.4, legend = c("Original (x)", "Generated (y)"), col = c("black", 'blue'), lty =1:2, lwd = 2) 

In [ ]:
summary_ = summary((x,y))
ks_ = ks_testD(x,y)
#dd = compare_distances(x,y)

To check numerically some first properties of the generated distribution, We output in the following table the skewness and kurtosis of both $x$ and $y$

In [ ]:
%%R
knitr::kable(head(py$summary_), caption = 'distributions characteristics', label = 'Ga1Dsum')

In [ ]:
%%R
knitr::kable(py$ks_, caption = 'KS Test', label = 'BiKS')

### N dimensional Examples

In [ ]:
(N,D) = (1000,2)
x = np.random.normal(size = (N, D))

Let us call the sampling function, filling up $y \in \mathbb{R}^{M\times 1}$.

In [ ]:
M = 1000
y = alg.sampler(x,M)

To check our results, let us compare both cdf of $x$ and $y$ in the following figure

<!--

In [ ]:
compare_plot(x,y,title = "original distribution (blue) versus generated distribution (yellow)")

-->

In [ ]:
%%R
plot(py$x, py$y, col = c(1,2), pch = 19, main = "original distribution (black) versus generated distribution (red)",
     ylab = "y", xlab = "x")

In [ ]:
summary_ = summary((x,y))
ks_ = ks_testD(x,y)
dd = compare_distances(x,y)

To check numerically some first properties of the generated distribution, We output in the following table the skewness and kurtosis of both $x$ and $y$

In [ ]:
%%R
knitr::kable(head(py$summary_), caption = 'distributions characteristics', label = 'Ga2Dsum')

In [ ]:
%%R
knitr::kable(py$ks_, caption = 'KS Test', label = 'Ga2DKS')

In [ ]:
%%R
knitr::kable(py$dd, caption = 'Hell. dist. / discrepancy err.', label = 'dicrepancy error')

#### ND t - distribution

In [ ]:
(N,D, df) = (1000,2,3)
x = np.random.standard_t(df,size = (N, D))

Let us call the sampling function, filling up $y \in \mathbb{R}^{M\times 1}$.

In [ ]:
M = 1000
y = alg.sampler(x,M)

To check our results, let us compare both cdf of $x$ and $y$ in the following figure
<!--

In [ ]:
compare_plot(x,y,title = "original distribution (blue) versus generated distribution (yellow)")

-->

In [ ]:
%%R
plot(py$x, py$y, col = c(1,2), pch = 19, main = "original distribution (black) versus generated distribution (red)",
     ylab = "y", xlab = "x" )

In [ ]:
summary_ = summary((x,y))
ks_ = ks_testD(x,y)
dd = compare_distances(x,y)

To check numerically some first properties of the generated distribution, We output in the following table the skewness and kurtosis of both $x$ and $y$

In [ ]:
%%R
knitr::kable(head(py$summary_), caption = 'distributions characteristics', label = 'Ga1Dsum')

In [ ]:
%%R
knitr::kable(py$ks_, caption = 'KS Test', label = 'St1DKS')

In [ ]:
%%R
knitr::kable(py$dd, caption = 'Hell. dist. / discrepancy err.', label = 'dicrepancy error')

#### Nd-Bimodal Gaussian distribution

In this section we study a bi-modal gaussian distribution.


In [ ]:
(N,D) = (1000,2)
x = np.random.normal(-5., 1., (int(N/2),D))
x = np.concatenate( (x,np.random.normal(+5., 1., (int(N/2),D))) )

Let us call the sampling function, filling up $y \in \mathbb{R}^{M\times 1}$, and let us plot both original and generated samples.

In [ ]:
M = 1000
y = alg.sampler(x,M)
compare_plot(x,y,title = "original distribution (blue) versus generated distribution (yellow)")

To check our results, let us compare both cdf of $x$ and $y$ in the following figure

In [ ]:
%%R
plot( py$x,py$y, pch = 19, main = "original distribution (black) versus generated distribution (red)",  ylab = "y", xlab = "x")

In [ ]:
summary_ = summary((x,y))
ks_ = ks_testD(x,y)
dd = compare_distances(x,y)

To check numerically some first properties of the generated distribution, We output in the following table the skewness and kurtosis of both $x$ and $y$

In [ ]:
%%R
knitr::kable(head(py$summary_), caption = 'distributions characteristics', label = 'Ga1Dsum')

In [ ]:
%%R
knitr::kable(py$ks_, caption = 'KS Test', label = 'St1DKS')

In [ ]:
%%R
knitr::kable(py$dd, caption = 'Hell. dist. / discrepancy err.', label = 'dicrepancy error')